In [1]:
import math
import sys
from collections import defaultdict
import itertools
from grammar import Pcfg

In [2]:
### Use the following two functions to check the format of your data structures in part 3 ###
def check_table_format(table):
    """
    Return true if the backpointer table object is formatted correctly.
    Otherwise return False and print an error.  
    """
    if not isinstance(table, dict): 
        sys.stderr.write("Backpointer table is not a dict.\n")
        return False
    for split in table: 
        if not isinstance(split, tuple) and len(split) ==2 and \
          isinstance(split[0], int)  and isinstance(split[1], int):
            sys.stderr.write("Keys of the backpointer table must be tuples (i,j) representing spans.\n")
            return False
        if not isinstance(table[split], dict):
            sys.stderr.write("Value of backpointer table (for each span) is not a dict.\n")
            return False
        for nt in table[split]:
            if not isinstance(nt, str): 
                sys.stderr.write("Keys of the inner dictionary (for each span) must be strings representing nonterminals.\n")
                return False
            bps = table[split][nt]            
            if isinstance(bps, str): # Leaf nodes may be strings
                continue 
            if not isinstance(bps, tuple):
                sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Incorrect type: {}\n".format(bps))
                return False
            if len(bps) != 2:
                sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Found more than two backpointers: {}\n".format(bps))
                return False
            for bp in bps: 
                if not isinstance(bp, tuple) or len(bp)!=3:
                    sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Backpointer has length != 3.\n".format(bp))
                    return False
                if not (isinstance(bp[0], str) and isinstance(bp[1], int) and isinstance(bp[2], int)):
                    print(bp)
                    sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a pair ((i,k,A),(k,j,B)) of backpointers. Backpointer has incorrect type.\n".format(bp))
                    return False
    return True

def check_probs_format(table):
    """
    Return true if the probability table object is formatted correctly.
    Otherwise return False and print an error.  
    """
    if not isinstance(table, dict): 
        sys.stderr.write("Probability table is not a dict.\n")
        return False
    for split in table: 
        if not isinstance(split, tuple) and len(split) ==2 and isinstance(split[0], int) and isinstance(split[1], int):
            sys.stderr.write("Keys of the probability must be tuples (i,j) representing spans.\n")
            return False
        if not isinstance(table[split], dict):
            sys.stderr.write("Value of probability table (for each span) is not a dict.\n")
            return False
        for nt in table[split]:
            if not isinstance(nt, str): 
                sys.stderr.write("Keys of the inner dictionary (for each span) must be strings representing nonterminals.\n")
                return False
            prob = table[split][nt]
            if not isinstance(prob, float):
                sys.stderr.write("Values of the inner dictionary (for each span and nonterminal) must be a float.{}\n".format(prob))
                return False
            if prob > 0:
                sys.stderr.write("Log probability may not be > 0.  {}\n".format(prob))
                return False
    return True

In [9]:
getHead = lambda x, grammar: set(rule[0] for rule in grammar.rhs_to_rules[x])
cartesian = lambda x, y: set([(i, j) for i in x for j in y])

class CkyParser(object):
    """ A CKY parser. """
    def __init__(self, grammar): 
        """ Initialize a new parser instance from a grammar. """
        self.grammar = grammar

    def getHead(self, x):
        return [rule[0] for rule in self.grammar.rhs_to_rules[x]]

    def is_in_language(self,tokens):
        """
        Membership checking. Parse the input tokens and return True if 
        the sentence is in the language described by the grammar. Otherwise
        return False
        """
        n = len(tokens)
        s = [[None]*(n+1) for i in range(n+1)]

        # Initialization
        for i in range(n):
            s[i][i+1] = set(self.getHead((tokens[i], )))

        # For each stage of the process
        for length in range(2, n+1):
            
            # For each generation of s[i][j]
            for i in range(n-length+1):
                j = i+length
                s[i][j] = set([])
                
                # Generate all posibilities of s[i][k], s[k][j]: i<k<j
                for k in range(i+1, j):

                    for pair in cartesian(s[i][k], s[k][j]):
                        s[i][j] = s[i][j].union(set(self.getHead(pair)))
        
        # Check if the entire sentence was generated by the parse
        if self.grammar.startsymbol in s[0][n]:
            return True

        return False
       
    def parse_with_backpointers(self, tokens):
        n = len(tokens)
        table = defaultdict(dict)
        probs = defaultdict(dict)
        
        # Initialization
        for i in range(n):
            for (lhs, rhs, prob) in self.grammar.rhs_to_rules[(tokens[i],)]:
                prob = math.log(prob)

                if lhs not in probs[(i,i+1)] or \
                    prob > probs[(i,i+1)][lhs]:
                    table[(i,i+1)][lhs] = tokens[i]
                    probs[(i,i+1)][lhs] = prob
                    
        # For each stage of the process
        for length in range(2, n+1):
            
            # For each generation of s[i][j]
            for i in range(n-length+1):
                j = i+length
                
                # Generate all posibilities of s[i][k], s[k][j]: i<k<j
                for k in range(i+1, j):
                    # Generate each pair
                    for B in table[(i, k)].keys():
                        for C in table[(k,j)].keys():

                            for(lhs, rhs, prob) in self.grammar.rhs_to_rules[(B,C)]:
                                prob = math.log(prob) + probs[(i,k)][B] + probs[(k,j)][C]
                                
                                if lhs not in probs[(i,j)] or \
                                    prob > probs[(i,j)][lhs]:
                                    table[(i,j)][lhs] = ((rhs[0], i, k), (rhs[1], k, j))
                                    probs[(i,j)][lhs] = prob

        # Check if the entire sentence was generated by the parse
        return table, probs

In [10]:
def get_tree(chart, i, j, nt): 
    """
    Return the parse-tree rooted in non-terminal nt and covering span i,j.
    """
    rhs = chart[(i,j)][nt]
    if isinstance(rhs, str):
        return (nt, rhs)
    else:
        return (
            nt,
            get_tree(chart, rhs[0][1], rhs[0][2], rhs[0][0]),
            get_tree(chart, rhs[1][1], rhs[1][2], rhs[1][0]))

In [11]:
if __name__ == "__main__":
    
    with open('atis3.pcfg','r') as grammar_file: 
        grammar = Pcfg(grammar_file) 
        parser = CkyParser(grammar)
        toks =['flights', 'from','miami', 'to', 'cleveland','.'] 
        print(parser.is_in_language(toks))
        
        table,probs = parser.parse_with_backpointers(toks)
        
        assert check_table_format(table)
        assert check_probs_format(probs)
        
        print(get_tree(table, 0, len(toks), grammar.startsymbol))

True
('TOP', ('NP', ('NP', 'flights'), ('NPBAR', ('PP', ('FROM', 'from'), ('NP', 'miami')), ('PP', ('TO', 'to'), ('NP', 'cleveland')))), ('PUN', '.'))


In [12]:
def print_tree(s):
    print(" ", list(range(len(s))))
    for n, row in enumerate(s):
        print(n, row[1:])

def print_table(s, n):
    x = [[None]*n for i in range(n)]
    for i in range(n):
        for j in range(i, n):
            x[i][j] = s.get((i,j), None)
    
    for i, row in enumerate(x):
        print()
        for j, data in enumerate(row):
#             if i>=j: continue
            print((i, j), data)

In [15]:
# evaluate_parser.py
import sys

def tokenize(line):
    tok = ''
    for c in line: 
        if c == " ":
            if tok: 
                yield tok
                tok = ""
        elif c == "(" or c==")":
            if tok: 
                yield tok
            yield c
            tok = ""
        else: 
            tok += c
    if tok: 
        yield tok
        tok = ""
           
def parse_tree(line):
    toks = tokenize(line)
    stack = []
    t = next(toks)
    try:
        while t:
            if t=="(":
                stack.append(t)
            elif t==")":
                subtree = []
                s = stack.pop()
                while s[0]!="(":
                    subtree.append(s)
                    s = stack.pop()
                stack.append(tuple(reversed(subtree)))
            else: 
                stack.append(t)
            t = next(toks)
    except StopIteration: 
        return stack.pop()
                

def get_leafs(tree):
    if isinstance(tree,str):
        return [tree]
    else: 
        result = []
        for x in tree[1:]:
            result.extend(get_leafs(x))
        return result
            

def get_constituents(tree,left=0):
    if not tree: 
        return [], left
    start = left
    if isinstance(tree,str): 
        return [],left+1
    else: 
        result = []
        phrase = tree[0]
        for subtree in tree[1:]:
            subspans, right = get_constituents(subtree, left)
            result.extend(subspans)
            left = right
        result.append((phrase,start,left))
        return result, left

def compute_parseval_scores(gold_tree, test_tree): 
    
    gold_const = set(get_constituents(gold_tree)[0])
    test_const = set(get_constituents(test_tree)[0])
    
    if not test_const: 
        return 0.0,0.0,0.0

    correct = len(gold_const.intersection(test_const))     
    recall = correct / float(len(gold_const))
    precision = correct / float(len(test_const))
    fscore = (2*precision*recall) / (precision+recall)
    return precision, recall, fscore 

def evaluate_parser(parser, treebank_file):
  
    total = 0
    unparsed = 0
    fscore_sum = 0.0
    for line in treebank_file:  
        gold_tree = parse_tree(line.strip())
        tokens = get_leafs(gold_tree)
        print("input: ",tokens)
        chart,probs = parser.parse_with_backpointers(tokens)
        print("target:    ",gold_tree)
        total += 1
        if not chart: 
            unparsed += 1
            res = tuple()
        else: 
            try:
                res = get_tree(chart,0,len(tokens),parser.grammar.startsymbol)
            except KeyError:
                unparsed += 1
                res = tuple() 
        print("predicted: ",res)
        #print(compute_parseval_scores(gold_tree, res))
        p,r,f = compute_parseval_scores(gold_tree, res)
        fscore_sum += f
        print("P:{} R:{} F:{}".format(p,r,f))
        print()
        
    parsed = total-unparsed 
    if parsed == 0:
        coverage = 0.0
        fscore_parsed = 0.0
        fscore_all = 0.0 
    else: 
        coverage =  (parsed / total) *100
        fscore_parsed = fscore_sum / parsed 
        fscore_all = fscore_sum / total
    print("Coverage: {:.2f}%, Average F-score (parsed sentences): {}, Average F-score (all sentences): {}".format(coverage, fscore_parsed, fscore_all))
        

with open('atis3.pcfg','r') as grammar_file, open('atis3_test.ptb','r') as test_file: 
    grammar = Pcfg(grammar_file) 
    parser = CkyParser(grammar)
    evaluate_parser(parser,test_file)

input:  ['flights', 'from', 'los', 'angeles', 'to', 'pittsburgh', '.']
target:     ('TOP', ('NP', ('NP', 'flights'), ('NPBAR', ('PP', ('FROM', 'from'), ('NP', ('LOS', 'los'), ('ANGELES', 'angeles'))), ('PP', ('TO', 'to'), ('NP', 'pittsburgh')))), ('PUN', '.'))
predicted:  ('TOP', ('NP', ('NP', 'flights'), ('NPBAR', ('PP', ('FROM', 'from'), ('NP', ('LOS', 'los'), ('ANGELES', 'angeles'))), ('PP', ('TO', 'to'), ('NP', 'pittsburgh')))), ('PUN', '.'))
P:1.0 R:1.0 F:1.0

input:  ['with', 'the', 'least', 'expensive', 'fare', '.']
target:     ('TOP', ('PP', ('WITH', 'with'), ('NP', ('THE', 'the'), ('NPBAR', ('ADJP', ('LEAST', 'least'), ('EXPENSIVE', 'expensive')), ('FARE', 'fare')))), ('PUN', '.'))
predicted:  ()
P:0.0 R:0.0 F:0.0

input:  ['flights', 'between', 'tampa', 'and', 'saint', 'louis', '.']
target:     ('TOP', ('NP', ('NP', 'flights'), ('PP', ('BETWEEN', 'between'), ('NP', ('NP', 'tampa'), ('NPBAR', ('AND', 'and'), ('NP', ('SAINT', 'saint'), ('LOUIS', 'louis')))))), ('PUN', '.'))
pre